# MSLS problem solution via linear programming 

Input: 
    .txt file with following shape:
    
    a11 a12 a13 ... a1m b1
    
    a21 a22 a23 ... a2m b2
    
    .
    .
    .
    
    an1 an2 an3 ... anm bn
    
    where aij, bk are integers
    
Output:
    vector of m rational numbers which presents solution of MSLS problem

In [1]:
import numpy as np
import pandas as pd
import itertools
from scipy.optimize import linprog
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def readFile(filename):
    if filename.endswith('.txt'):
        with open(filename, 'r') as f:
            mat, A, b = [], [], []
            for line in f:
                eq = []
                for num in line.split():
                    eq.append(int(num))
                A.append(eq[:-1])
                b.append(eq[-1])
                mat.append(eq)
    else:
        raise Exception("We need .txt file :)")
    return mat, A, b

In [4]:
def checkGroup(A, b):
    c = np.zeros(shape=(1, len(A[0])))
    sol = linprog(c = c, A_eq=A, b_eq= b, method = 'simplex')
    return sol.success, sol.x

In [5]:
def comb_with_excludes(lst, n, excludes, i=0, taken=[]):
    if n == 0:
        yield taken  # no more needed
    elif i <= len(lst) - n:
        t2 = taken + [lst[i],]  # add current element
        if not any(e.issubset(t2) for e in excludes):
            yield from comb_with_excludes(lst, n-1, excludes, i+1, t2)
        if i < len(lst) - n:  # skip current element
            yield from comb_with_excludes(lst, n, excludes, i+1, taken)

In [6]:
# comb_with_excludes([1,2,3,4, 5], 3, [{1, 2}, {2, 3}])

Algorithm with partitioning

In [7]:
def MSLS2(filepath):
    print("-----------------------------------------------------------------")
    print("FILE: ", filepath)
    print("WORKING ON IT ...")
    mat, _, _ = readFile(filepath)
    eqNums = [i for i in range(len(mat))]
    
    excluded = []
    
    maxSatisfied = 1
    resultX = linprog(c = [0 for _ in range(len(mat[0]) - 1)], 
                      A_eq=[mat[0][:-1]], 
                      b_eq= [mat[0][-1]]
                     )
    maxSatisfied = 0
    finalComb = []
    for k in range(2, len(mat)+1):
        eq_comb = list(comb_with_excludes([e for e in range(len(mat))], k, excluded))
        
        for comb in eq_comb:
            system = np.array([mat[e] for e in comb])
            
            A_tmp, b_tmp = system[0:, :-1], system[0:, -1]
            
            success, sol = checkGroup(A_tmp, b_tmp)
                
            if success == True:
                maxSatisfied = k
                finalComb = comb
                finalSol = sol
            else:
                excluded.append(set(comb))
                
    print("Number of satisfied eq is: ", maxSatisfied)
    print("Solution x: ", finalSol)
    print("-----------------------------------------------------------------")

In [8]:
def getFiles(dir_path):
    fileNames = []

    for file_path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, file_path)):
            fileNames.append(dir_path + '/' + file_path)
    return fileNames

# Tests

In [10]:
MSLS2('tests/25x5.txt')

-----------------------------------------------------------------
FILE:  tests/25x5.txt
WORKING ON IT ...
Number of satisfied eq is:  5
Solution x:  [5.02117154 0.73970634 8.48604109 2.65616595 0.36797689]
-----------------------------------------------------------------


In [9]:
fileNames = getFiles('tests')
for fileName in fileNames:
    MSLS2(fileName)

-----------------------------------------------------------------
FILE:  tests/25x5.txt
WORKING ON IT ...
Number of satisfied eq is:  5
Solution x:  [5.02117154 0.73970634 8.48604109 2.65616595 0.36797689]
-----------------------------------------------------------------
-----------------------------------------------------------------
FILE:  tests/39x4.txt
WORKING ON IT ...
Number of satisfied eq is:  6
Solution x:  [0.4906705  0.9028782  0.22008648 0.41947005]
-----------------------------------------------------------------
-----------------------------------------------------------------
FILE:  tests/38x4.txt
WORKING ON IT ...
Number of satisfied eq is:  7
Solution x:  [12.2834661   5.70204027  1.34613433  2.84878793]
-----------------------------------------------------------------
-----------------------------------------------------------------
FILE:  tests/100x50.txt
WORKING ON IT ...


KeyboardInterrupt: 